In [55]:
"""
오버라이딩 디스크립터와 논오버라이딩 디스크립터

- 파이썬 속성 처리 -> 비대칭성이 있음 
- 객체에 속성이 있으면 해당 속성을 읽지만, 속성이 없으면 클래스 속성을 읽음
- 객체에 속성 할당하면, 클래스에는 영향 없음 
"""

# 출력을 위한 보조함수 
def cls_name(obj_or_cls) : 
    cls = type(obj_or_cls)
    if cls is type : 
        cls = obk_or_cls
        
    return cls.__name__.split('.')[-1]

def display(obj) : 
    cls = type(obj)
    if cls is type : 
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int] : 
        return repr(obj)    
    else : 
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args) : 
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))
    
class Overriding : 
    """데이터 디스크립터 / 강제 디스크립터"""
    
    def __get__(self, instance, owner) : 
        print_args('get', self, instance, owner)
        
    def __set__(self, instance, value) : 
        print_args('set', self, instance, value)
        
class OverridingNoGet : 
    """``__get__()``이 없는 오버라이딩 디스크립터"""
    
    def __set__(self, instance, value) :
        print_args('set', self, instance, value)
        
class NonOverriding : 
    """비데이터 디스크립터 / 가릴 수 있는 디스크립터"""
    
    def __get__(self, instance, owner) : 
        print_args('get', self, instance, owner)
        
class Managed : 
    # 클래스 속성 (cv)
    over = Overriding() # getter / setter
    over_no_get = OverridingNoGet() # setter
    non_over = NonOverriding() # getter
    
    def spam(self) : 
        print('-> Managed.spam({})'.format(display(self)))

In [56]:
"""
- __set__() 메서드를 구현한 디스크립터를 '오버라이딩 디스크립터'
- 객체 속성에 할당하려는 시도를 가로챔 
"""

obj = Managed()
obj.over # 오버라이딩 디스크립터 객체

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [57]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [58]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [59]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [60]:
obj.__dict__['over'] = 8
vars(obj)

{'over': 8}

In [61]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [62]:
obj.over_no_get

In [63]:
Managed.over_no_get

In [64]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [65]:
obj.over_no_get

In [66]:
obj.__dict__['over_no_get'] = 9

In [67]:
obj.over_no_get

9

In [68]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [69]:
obj.over_no_get

9

In [70]:
obj = Managed()
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [71]:
obj.non_over = 7

In [72]:
obj.non_over

7

In [73]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [74]:
del obj.non_over

In [75]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [76]:
obj = Managed()
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x7fabf9e3fac0>>

In [79]:
dir(Managed)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'non_over',
 'over',
 'over_no_get',
 'spam']

In [80]:
obj.spam = 7

In [81]:
obj.spam

7

In [83]:
obj.__dict__

{'spam': 7}

In [84]:
Managed.spam

<function __main__.Managed.spam(self)>

In [97]:
import collections

class Text(collections.UserString) : 
    def __repr__(self) : 
        return 'Text({!r})'.format(self.data)
    
    def reverse(self) : 
        return self[::-1]
    
word = Text('forward')

In [98]:
word

Text('forward')

In [99]:
word.__dict__

{'data': 'forward'}

In [100]:
Text.__dict__

mappingproxy({'__module__': '__main__',
              '__repr__': <function __main__.Text.__repr__(self)>,
              'reverse': <function __main__.Text.reverse(self)>,
              '__doc__': None,
              '__abstractmethods__': frozenset(),
              '_abc_impl': <_abc._abc_data at 0x7fabd89feb80>})

In [101]:
word.reverse()

Text('drawrof')

In [102]:
Text.reverse(Text('backward'))

Text('drawkcab')

In [103]:
type(Text.reverse), type(word.reverse)

(function, method)

In [106]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')]))

['diaper', (30, 20, 10), Text('desserts')]

In [107]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

In [108]:
Text.reverse.__get__(None, Text)

<function __main__.Text.reverse(self)>

In [109]:
word.reverse.__self__

Text('forward')

In [110]:
word.reverse.__func__ is Text.reverse

True